# Project 3


## RDS Database ETL
### 1. Getting data from the RDS database

In [3]:
from database_utils import DatabaseConnector

## read credentials
connector_rds = DatabaseConnector()
ai_core_yaml = 'db_creds.yaml'
connector_rds.read_db_creds(ai_core_yaml)

## initialising and returning an sqlalchemy database engine.
connection_rds = connector_rds.init_db_engine()

## Reading the data from the RDS database
connector_tables_rds = connector_rds.list_db_tables()





Credentials loaded sucessfully!!
Connection successful!
Tables in the database: ['legacy_store_details', 'dim_card_details', 'legacy_users', 'orders_table']


### 2. Extracting the table to a pandas DataFrame.

In [4]:
from data_extraction import DataExtractor
user_table_rds = 'legacy_users'

extractor_rds = DataExtractor(connection_rds)

user_df_rds = extractor.read_rds_table(user_table_rds)
print(user_df_rds.head(5))
print(user_df_rds.info())




   index first_name last_name date_of_birth                    company  \
0      0   Sigfried     Noack    1990-09-30         Heydrich Junitz KG   
1      1        Guy     Allen    1940-12-01                    Fox Ltd   
2      2      Harry  Lawrence    1995-08-02  Johnson, Jones and Harris   
3      3     Darren   Hussain    1972-09-23                Wheeler LLC   
4      4      Garry     Stone    1952-12-20                 Warner Inc   

                  email_address  \
0             rudi79@winkler.de   
1  rhodesclifford@henderson.com   
2  glen98@bryant-marshall.co.uk   
3    daniellebryan@thompson.org   
4       billy14@long-warren.com   

                                             address         country  \
0                       Zimmerstr. 1/0\n59015 Gießen         Germany   
1  Studio 22a\nLynne terrace\nMcCarthymouth\nTF0 9GH  United Kingdom   
2                 92 Ann drive\nJoanborough\nSK0 6LR  United Kingdom   
3             19 Robinson meadow\nNew Tracy\nW22 2QG  Un

### Cleaning data

In [5]:
from data_cleaning import DataCleaning

clean_data_rds = DataCleaning()
clean_df_rds = clean_data_rds.clean_user_data(user_df_rds)



Checking for Null values:
index            0.0
first_name       0.0
last_name        0.0
date_of_birth    0.0
company          0.0
email_address    0.0
address          0.0
country          0.0
country_code     0.0
phone_number     0.0
join_date        0.0
user_uuid        0.0
dtype: float64
Deleting rows with more than 4 Null values
       index first_name last_name date_of_birth                       company  \
0          0   Sigfried     Noack    1990-09-30            Heydrich Junitz KG   
1          1        Guy     Allen    1940-12-01                       Fox Ltd   
2          2      Harry  Lawrence    1995-08-02     Johnson, Jones and Harris   
3          3     Darren   Hussain    1972-09-23                   Wheeler LLC   
4          4      Garry     Stone    1952-12-20                    Warner Inc   
...      ...        ...       ...           ...                           ...   
15315  14913    Stephen   Jenkins    1943-08-09  Thornton, Carroll and Newman   
15316  14994    

### 3. Loading the data
#### uploading the cleaned dataframe into the local database


In [6]:
# Connect with the local database

#read creds
uploader = DatabaseConnector()
ml_yaml = 'ml_dbs.yaml'
uploader.read_db_creds(ml_yaml)

# initialising and returning an sqlalchemy database engine.

uploading = uploader.init_db_engine_postgresql()

#load clean rds dataframe into local database with a new table
uploader.upload_to_db(uploading, clean_df, 'dim_users')





Credentials loaded sucessfully!!
Connection successful!
Data uploaded successfully to dim_users table.


## PDF data: ETL
### 1.Getting the data from PDF & load in df

In [7]:
import tabula

connector_pdf = DatabaseConnector()
extractor_pdf = DataExtractor(connector_pdf)

link = "https://data-handling-public.s3.eu-west-1.amazonaws.com/card_details.pdf"
extraction_pdf = extractor_pdf.retrieve_pdf_data(link)
print(extraction_pdf)

Failed to import jpype dependencies. Fallback to subprocess.
No module named 'jpype'
'pages' argument isn't specified.Will extract only from page 1 by default.


[            card_number expiry_date                card_provider  \
0        30060773296197       09/26  Diners Club / Carte Blanche   
1       349624180933183       10/23             American Express   
2      3529023891650490       06/23                 JCB 16 digit   
3       213142929492281       09/27                 JCB 15 digit   
4          502067329974       10/25                      Maestro   
5      3506661913512980       11/23                 JCB 16 digit   
6       377549437870679       07/27             American Express   
7      2321541881278150       02/29                   Mastercard   
8      6011037917693140       02/24                     Discover   
9          502049986008       07/23                      Maestro   
10     6501622116456490       10/28                     Discover   
11     6502495513721380       11/27                     Discover   
12     3551071736826240       11/31                 JCB 16 digit   
13      371493449732930       10/29            

### 2. put into a dataframe

In [8]:

pdf_df = extractor_pdf.load_pdf_df(extraction_pdf)

TypeError: DataExtractor.__init__() missing 1 required positional argument: 'connector'

In [ ]:
clean_pdf = clean_data.clean_card_data(pdf_data_extraction)


#DBConnector
# read creds
#return db engine
#read data 

#dataextractor
#put into a df

#cleaning
#DB COnnector
#Connect with local datbase (uploader and uploading)

Checking for Null values:


AttributeError: 'list' object has no attribute 'isna'